### Preliminary submission looking at basic inter-likeness of clikcing an ad

thought list:
- how many times has an ad appeared?
- how many times has an ad been clicked?
- % of time the ad was clicked vs appeared
- the more an ad appears the higher chance it has of being clicked (as a baseline truth, not thinking about locale, user diversity, etc)
- can I generate a weight to rank each ad based on the above properties?
- if so we can use this to sort the ads and group by display_id
- This could provide a fair baseline based soley on click-by-show popularity not taking into account the other data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
import collections
import json
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#train = pd.read_csv("data/clicks_train.csv")
temp_train = pd.read_csv("data_mini/clicks_train_mini.csv")

In [ ]:
def df_info(df):
    print 'temp_train: ', df.shape
    print 'Num of unique display_id: ', len(df['display_id'].unique())
    print 'Num of unique ad_id: ', len(df['ad_id'].unique())
    print 'Click ratio: {}% of ads clicked.'.format(round(float(df['clicked'].value_counts()[1])/df.shape[0]*100,3))
    display(df.head())

df_info(temp_train)

In [ ]:
temp_train_copy = temp_train.copy()
# value = temp_train_copy.groupby('ad_id').count()
# value

display(temp_train_copy.loc[temp_train_copy['ad_id'] == 347571].shape[0])
display(temp_train_copy.loc[temp_train_copy['ad_id'] == 347571].loc[temp_train_copy['clicked'] == 1].shape[0])
4.0/12

In [ ]:
# could add here a normalizer value to account for ads with very low and very high occurances.

# VERY SLOW... c'mon we can make this better.... took 20 sec

#gives me a dict where the key is the ad_id and the value is a tuple with information about its click ratio
unique_ads = temp_train['ad_id'].unique().tolist()
ad_count_dict = {}
for ad in unique_ads:
#     temp_tuple = () #3 elements for clicked, not clicked, and %
    
    # KYLE: Next line just keeps the part of the table that we want
    # Maybe it is faster??
    temp = temp_train_copy.loc[temp_train_copy['ad_id'] == ad]
    
    appeared = temp.shape[0]
    clicked = temp.loc[temp_train_copy['clicked'] == 1].shape[0]
    not_clicked = temp.loc[temp_train_copy['clicked'] == 0].shape[0]
    percent_clicked = float(clicked)/appeared
    
    temp_tuple = (percent_clicked, appeared, clicked, not_clicked)
    
    
    ad_count_dict[ad] = temp_tuple
ad_count_dict
    

In [ ]:
#gives me a dict where the key is the display_id and the value is a list of all the ad_ids associated with it

display_list = temp_train['display_id'].unique().tolist()
display_dict = {}
for display in display_list:
    ad_list = temp_train.loc[temp_train['display_id'] == display]['ad_id'].tolist()
    
    display_dict[display] = ad_list
display_dict

In [ ]:
# sorted lists of ad_id based on click probability binded to a key of display_id

display_dict_sorted = {}
for key, value in display_dict.iteritems():
    temp_list = []
    for e in value:
        temp_list.append(ad_count_dict[e][0])
    
    sorted_list = [x for (y,x) in sorted(zip(temp_list, value), reverse=True)]
    sorted_list = map(str, sorted_list) #turn whole list to string
    
    display_dict_sorted[key] = key, ' '.join(sorted_list) #strip out everything and join with ' '.

display_dict_sorted

In [ ]:
submit_df = pd.DataFrame.from_dict(display_dict_sorted, orient='index')
submit_df.columns = ['display_id', 'ad_id']
submit_df.set_index(['display_id'], inplace=True)
# display(submit_df.head())
submit_df.to_csv('csv_submissions/prelim_model_v1.csv')


### Train on full train file

In [2]:
train = pd.read_csv("data/clicks_train.csv")

In [21]:
from timeit import default_timer as timer

unique_ads = train['ad_id'].unique().tolist()
ad_count_dict = {}
i = 0
ct = len(unique_ads)

start = timer()

for ad in unique_ads:
#     temp_tuple = () #3 elements for clicked, not clicked, and %
    
    temp = train.loc[train['ad_id'] == ad]
    
    appeared = temp.shape[0]
    clicked = temp.loc[train['clicked'] == 1].shape[0]
    not_clicked = temp.loc[train['clicked'] == 0].shape[0]
    
#     appeared = train.loc[train['ad_id'] == ad].shape[0]
#     clicked = train.loc[train['ad_id'] == ad].loc[train['clicked'] == 1].shape[0]
#     not_clicked = train.loc[train['ad_id'] == ad].loc[train['clicked'] == 0].shape[0]
    
    percent_clicked = float(clicked)/appeared
    
    temp_tuple = (percent_clicked, appeared, clicked, not_clicked)
    
    
    ad_count_dict[ad] = temp_tuple
    
    if ((i % 1000) == 0):
        elapsed = (timer() - start) / 3600.0
        rate = elapsed / (i+1)
        print('Ad #{} of {}, Time left = {:,.2f} hours'.format(i+1, ct, (ct - i)*rate))
        
    i += 1
    


Ad #1 of 478950, Time left = 45.46 hours
Ad #101 of 478950, Time left = 45.65 hours
Ad #201 of 478950, Time left = 45.58 hours
Ad #301 of 478950, Time left = 45.58 hours
Ad #401 of 478950, Time left = 45.52 hours
Ad #501 of 478950, Time left = 45.47 hours


KeyboardInterrupt: 

In [ ]:
display_list = train['display_id'].unique().tolist()
display_dict = {}
for display in display_list:
    ad_list = train.loc[train['display_id'] == display]['ad_id'].tolist()
    
    display_dict[display] = ad_list

    
# step 3
    
display_dict_sorted = {}
for key, value in display_dict.iteritems():
    temp_list = []
    for e in value:
        temp_list.append(ad_count_dict[e][0])
    
    sorted_list = [x for (y,x) in sorted(zip(temp_list, value), reverse=True)]
    sorted_list = map(str, sorted_list) #turn whole list to string
    
    display_dict_sorted[key] = key, ' '.join(sorted_list) #strip out everything and join with ' '.


    
#step 4
    
submit_df = pd.DataFrame.from_dict(display_dict_sorted, orient='index')
submit_df.columns = ['display_id', 'ad_id']
submit_df.set_index(['display_id'], inplace=True)
submit_df.to_csv('csv_submissions/prelim_model_v1.csv')